In [1]:
from net import skip
from net.losses import ExclusionLoss, plot_image_grid, StdLoss, CannyDiceLoss
from net.noise import get_noise
from utils.image_io import *
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from collections import namedtuple
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
from torchsummary import summary

import os

import matplotlib.pyplot as plt
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pad = 'reflection'
net = skip(
    3, 3,
    num_channels_down=[32, 64, 128, 256, 512],
    num_channels_up=[32, 64, 128, 256, 512],
    num_channels_skip=[8, 8, 8, 8, 8],
    upsample_mode='bilinear',
    filter_size_down=5,
    filter_size_up=5,
    need_sigmoid=False, need_bias=True, pad=pad, act_fun='LeakyReLU').to(device)

In [2]:
summary(net, input_size=(3, 480, 640), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ReflectionPad2d-1           [1, 3, 480, 640]               0
            Conv2d-2           [1, 8, 480, 640]              32
       BatchNorm2d-3           [1, 8, 480, 640]              16
         LeakyReLU-4           [1, 8, 480, 640]               0
   ReflectionPad2d-5           [1, 3, 484, 644]               0
            Conv2d-6          [1, 32, 240, 320]           2,432
       BatchNorm2d-7          [1, 32, 240, 320]              64
         LeakyReLU-8          [1, 32, 240, 320]               0
   ReflectionPad2d-9          [1, 32, 244, 324]               0
           Conv2d-10          [1, 32, 240, 320]          25,632
      BatchNorm2d-11          [1, 32, 240, 320]              64
        LeakyReLU-12          [1, 32, 240, 320]               0
  ReflectionPad2d-13          [1, 32, 240, 320]               0
           Conv2d-14           [1, 8, 2